# テキストマイニングによるTwitter個人アカウントの性格推定

- 2018-09-17 PyCon JP 2018
- @jumpyoshim / Jumpei Yoshimura
- iRidge, Inc. - Gold Sponsor

# 目次
- ツイートの収集
- 文書分類器の作成
  - 形態素解析
  - 特徴ベクトル
  - 機械学習

# ツイートの収集

In [7]:
import twitter
from ipython_secrets import get_secret

CONSUMER_KEY = get_secret('CONSUMER_KEY')
CONSUMER_SECRET = get_secret('CONSUMER_SECRET')
ACCESS_TOKEN_KEY = get_secret('ACCESS_TOKEN_KEY')
ACCESS_TOKEN_SECRET = get_secret('ACCESS_TOKEN_SECRET')

api = twitter.Api(consumer_key=CONSUMER_KEY,
                              consumer_secret=CONSUMER_SECRET,
                              access_token_key=ACCESS_TOKEN_KEY,
                              access_token_secret=ACCESS_TOKEN_SECRET)

api.GetUserTimeline(screen_name='PyConJ', count=2)

[Status(ID=1039065269425983488, ScreenName=PyConJ, Created=Mon Sep 10 08:17:19 +0000 2018, Text='RT @smarttradeinc: PyConJP2018のノベルティグッズが出来上がりました！皆さん是非ブースにお立ち寄りください♪\n #pyconjp https://t.co/6BGHJkCAXY'),
 Status(ID=1039064907965030401, ScreenName=PyConJ, Created=Mon Sep 10 08:15:53 +0000 2018, Text='RT @yuzutas0: 昨年登壇したけど、最高のイベントでした。オススメです。実はベストトークアワード優秀賞を頂いたりもしました。 #pyconjp\n\n「PyCon JP 2018 ひろがるPython」登壇者座談会 ～わたしのPythonのひろげかた～：CodeZine（…')]

# 形態素解析

In [2]:
import MeCab

tagger = MeCab.Tagger('mecabrc')
data = []
for tweet in tweets:
    node = tagger.parseToNode(tweet.text)
    words = []
    while node:
        meta = node.feature.split(',')
        if meta[0] == '名詞':
            words.append(node.surface.lower())
        node = node.next
    data.append(words)
print(data)

[['rt', '@', 'smarttradeinc', ':', 'pyconjp', '2018', 'ノベルティグッズ', '皆さん', '是非', 'ブース', '♪', '#', 'pyconjp', 'https', '://', 't', '.', 'co', '/', '6', 'bghjkcaxy'], ['rt', '@', 'yuzutas', '0', ':', '昨年', '登壇', '最高', 'イベント', 'オススメ', 'ベスト', 'トーク', 'アワード', '優秀', '賞', '#', 'pyconjp', 'pycon', 'jp', '2018', 'python', '登壇', '者', '座談', '会', '～', 'わたし', 'python', 'かた', '～：', 'codezine']]


# 特徴ベクトル

In [3]:
from gensim import corpora, matutils

dictionary = corpora.Dictionary(data)
data_train = []
for datum in data:
    bow = dictionary.doc2bow(datum)
    dense = list(matutils.corpus2dense([bow], num_terms=len(dictionary)).T[0])
    data_train.append(dense)
print(data_train)

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0]]


# 機械学習

In [6]:
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()
label_train = ['aaaaa', 'bbbbb']
estimator.fit(data_train, label_train)
estimator.predict(data_train)

array(['aaaaa', 'bbbbb'], dtype='<U5')